<a href="https://colab.research.google.com/github/dhivyapm/divya/blob/master/Random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I found the information about how the model works in this website
https://medium.com/coinmonks/what-is-entropy-and-why-information-gain-is-matter-4e85d46d2f01

In [5]:
from google.colab import files
upload = files.upload()

Saving satelite.csv to satelite.csv


In [6]:
import pandas as pd # to read the dataset and convert the dataset to DataFrame which is in table format
import numpy as np # numpy is for computing, here I'm using this package for randomly select the rows from the dataset
import random # to randomly select the datapoints from dataset
df = pd.read_csv('/content/satelite.csv')
print(df.shape)

(6435, 37)


In [11]:
features = ['Column17','Column18','Column19','Column20']
random.seed(30) # If you want to generate the same number every time,before doing the shuffle
df = df.sample(frac=1)# shuffle the dataset
df.shape

(6435, 37)

In [12]:
#train_test_Split
train_size = int(0.9 * len(df)) #train_size =5791
X_train = df[features][:train_size] # X_train takes the values of the columns from 1 to 36 from the dataset upto the train_size rows(5791 rows)
y_train = df['Column37'][:train_size] #y_train takes the last column values which is label upto the train_size(5791 rows)
X_test = df[features][train_size:] # X_test takes the values of the columns from 1 to 36 starting from rows 5791 upto the dataset length which is 6435
y_test = df['Column37'][train_size:] # y_test takes the last column values which is label starting from rows 5791 upto the dataset length which is 6435
print(X_train)

      Column17  Column18  Column19  Column20
474         47        31       131       142
3789        72        85        94        79
3255        74        79        84        66
4638        57        55        78        72
551         78        91        96        78
...        ...       ...       ...       ...
33          79       107       109        87
3873        67        72        70        50
6217        63        79        85        80
6379        88       103       113        92
30          68        75        79        67

[5791 rows x 4 columns]


In [0]:
#calculating the bootstrap that is taking sample from train set and left out samples in dataset will be taken as out-of-bag which is used to test the model performance
def n_bootstrap(X_train, y_train):
    bootstrap_indices = list(np.random.choice(range(len(X_train)), len(X_train), replace = True)) #it as the index of the sample selected from the training set
    oob_indices = [i for i in range(len(X_train)) if i not in bootstrap_indices] # it as the index of the left over samples in the training set
    X_bootstrap = X_train.iloc[bootstrap_indices].values # here it takes the values of the specific index from the training
    y_bootstrap = y_train.iloc[bootstrap_indices].values # it takes the array elements of the specific indices(labels)
    X_oob = X_train.iloc[oob_indices].values # it takes the values of the left over samples in the training set
    y_oob = y_train.iloc[oob_indices].values # it takes the array elements from the left over samples in the training set(labels)
    return X_bootstrap, y_bootstrap, X_oob, y_oob # returns all the values of bootstrap and oob

In [39]:
X_bootstrap,y_bootstrap,X_oob,y_oob = n_bootstrap(X_train,y_train)
print('X_bootstrap:\n',X_bootstrap,'\ny_bootstrap:\n',y_bootstrap)
print('X_oob:\n',X_oob,'\ny_oob:',y_oob)

X_bootstrap:
 [[ 63  79  85  80]
 [101 116 122  96]
 [ 75  88  90  72]
 ...
 [ 74  87  92  66]
 [ 59  51  79  72]
 [ 71 111 118  96]] 
y_bootstrap:
 [1 3 4 ... 4 5 1]
X_oob:
 [[ 74  79  84  66]
 [ 57  55  78  72]
 [ 78  91  96  78]
 ...
 [ 86 109 112  89]
 [ 67  66  72  53]
 [ 47  37 111 105]] 
y_oob: [7 5 4 ... 4 7 2]


In [0]:
# calculate the Entropy(measure of disorder)
#To which group does this sample belongs to
#Entropy controls how a Decision Tree decides to split the data.
#where entropy takes in a probability of a class within a node 
def entropy(p):
    if p == 0:
        return 0
    elif p == 1:
        return 0
    else:
        return  - (p * np.log2(p) + (1 - p) * np.log2(1-p))
         # these formula provided in these https://medium.com/coinmonks/what-is-entropy-and-why-information-gain-is-matter-4e85d46d2f01
         # in that they clearly explained the concept of entropy and Information-Gain

In [0]:
#calculate the information gain to compare the entropy before and after split
#Information_gain takes in a list of the classes from the left and right child and returns the information gain of that particular split.
def information_gain(left_child, right_child):
    parent = left_child + right_child
    p_parent = parent.count(1) / len(parent) if len(parent) > 0 else 0
    p_left = left_child.count(1) / len(left_child) if len(left_child) > 0 else 0
    p_right = right_child.count(1) / len(right_child) if len(right_child) > 0 else 0
    IG_p = entropy(p_parent) #calculate the entropy of parent node
    IG_l = entropy(p_left) #calculate the entropy of left child node
    IG_r = entropy(p_right) #calculate the entropy of right child node
    return IG_p - len(left_child) / len(parent) * IG_l - len(right_child) / len(parent) * IG_r #return the information gain of parent and childs

In [0]:
#function for finding the split point for the decision-tree based on the highest information gain
#select random features from the bootstrap 
#for each feature selected, iterate through each value in the X_bootstrap and compute the information gain
#Return a array of values that gives the highest information gain, which will represent a node in a tree consist of 
#index of the feature,split-value,left-child and right-child node.
def find_split_point(X_bootstrap, y_bootstrap, max_features):
    feature_ls = list()
    num_features = len(X_bootstrap[0])
    
    while len(feature_ls) <= max_features:
        feature_idx = random.sample(range(num_features), 1)
        if feature_idx not in feature_ls:
            feature_ls.extend(feature_idx)
    
    best_info_gain = -999
    node = None
    for feature_idx in feature_ls:
        for split_point in X_bootstrap[:,feature_idx]:
            left_child = {'X_bootstrap': [], 'y_bootstrap': []}
            right_child = {'X_bootstrap': [], 'y_bootstrap': []}
            
            # split children for continuous variables
            if type(split_point) in [int, float]:
                for i, value in enumerate(X_bootstrap[:,feature_idx]):
                    if value <= split_point:
                        left_child['X_bootstrap'].append(X_bootstrap[i])
                        left_child['y_bootstrap'].append(y_bootstrap[i])
                    else:
                        right_child['X_bootstrap'].append(X_bootstrap[i])
                        right_child['y_bootstrap'].append(y_bootstrap[i])
                  # split children for categoric variables
            else:
                for i, value in enumerate(X_bootstrap[:,feature_idx]):
                    if value == split_point:
                        left_child['X_bootstrap'].append(X_bootstrap[i])
                        left_child['y_bootstrap'].append(y_bootstrap[i])
                    else:
                        right_child['X_bootstrap'].append(X_bootstrap[i])
                        right_child['y_bootstrap'].append(y_bootstrap[i])
                
            split_info_gain = information_gain(left_child['y_bootstrap'], right_child['y_bootstrap'])
            if split_info_gain > best_info_gain:
                best_info_gain = split_info_gain
                left_child['X_bootstrap'] = np.array(left_child['X_bootstrap'])
                right_child['X_bootstrap'] = np.array(right_child['X_bootstrap'])
                node = {'information_gain': split_info_gain, 
                        'left_child': left_child, 
                        'right_child': right_child, 
                        'split_point': split_point,
                        'feature_idx': feature_idx}
                
    
    return node            

In [0]:
node = find_split_point(X_bootstrap,y_bootstrap,max_features=2)

In [53]:
np.array(node['right_child'])

array({'X_bootstrap': array([[ 63,  79,  85,  80],
       [ 75,  88,  90,  72],
       [ 56,  49, 108, 100],
       ...,
       [ 88, 103, 108,  81],
       [ 74,  87,  92,  66],
       [ 59,  51,  79,  72]]), 'y_bootstrap': [1, 4, 2, 5, 1, 4, 1, 5, 5, 3, 4, 7, 3, 7, 7, 1, 2, 1, 3, 3, 1, 5, 3, 3, 7, 4, 2, 5, 1, 5, 7, 3, 7, 5, 2, 5, 3, 7, 3, 2, 1, 3, 3, 3, 7, 3, 3, 5, 7, 1, 5, 3, 3, 7, 1, 7, 4, 7, 1, 5, 3, 1, 7, 1, 3, 7, 5, 1, 1, 7, 3, 3, 7, 3, 2, 1, 2, 7, 3, 2, 4, 1, 1, 1, 5, 5, 7, 7, 5, 1, 3, 3, 1, 4, 4, 1, 1, 4, 1, 2, 5, 1, 3, 5, 3, 5, 1, 2, 7, 2, 3, 3, 7, 2, 1, 2, 1, 7, 2, 7, 3, 7, 5, 1, 4, 1, 5, 1, 7, 2, 3, 3, 2, 7, 2, 3, 3, 2, 2, 7, 4, 1, 4, 7, 5, 3, 7, 4, 1, 5, 2, 2, 1, 1, 4, 4, 3, 2, 5, 3, 4, 4, 7, 4, 4, 1, 3, 7, 3, 7, 7, 3, 7, 5, 7, 7, 7, 7, 1, 1, 2, 4, 2, 2, 7, 3, 3, 1, 2, 3, 2, 3, 1, 1, 3, 7, 7, 1, 3, 1, 4, 3, 1, 3, 4, 1, 4, 7, 2, 2, 5, 1, 7, 3, 7, 5, 1, 4, 7, 1, 7, 7, 5, 7, 3, 4, 4, 5, 3, 7, 5, 5, 1, 1, 7, 4, 1, 7, 7, 4, 7, 3, 2, 3, 2, 1, 3, 2, 1, 3, 4, 7, 5, 5, 3, 7, 7, 7, 

In [54]:
np.array(node['left_child'])

array({'X_bootstrap': array([[101, 116, 122,  96],
       [ 99, 118, 122,  96],
       [101, 116, 122,  96],
       [ 88, 107, 118,  96],
       [ 64, 107, 122,  96],
       [ 68, 112, 128,  96],
       [ 71, 107, 118,  96],
       [ 71, 107, 118,  96],
       [ 64, 107, 118,  96],
       [ 71, 103, 113,  96],
       [ 63,  64, 104,  96],
       [ 75, 103, 113,  96],
       [ 90, 113, 122,  96],
       [ 64, 112, 122,  96],
       [ 67, 107, 123,  96],
       [ 63,  64, 104,  96],
       [ 93, 111, 118,  96],
       [ 88, 116, 122,  96],
       [ 60, 107, 122,  96],
       [ 59,  67, 104,  96],
       [ 80, 107, 118,  96],
       [ 67, 107, 123,  96],
       [ 67, 111, 118,  96],
       [ 71, 111, 123,  96],
       [ 71, 111, 118,  96],
       [ 78, 113, 112,  96],
       [ 67, 111, 123,  96],
       [ 53,  45,  96,  96],
       [ 71, 107, 113,  96],
       [ 67, 107, 113,  96],
       [ 92, 116, 122,  96],
       [ 71, 103, 118,  96],
       [ 75, 107, 123,  96],
       [ 67, 107, 118

In [55]:
np.array(node['feature_idx'])

array(3)

In [56]:
node['split_point']

96

In [57]:
node['information_gain']

0.02157664002847881